In [ ]:
import pandas as pd
import csv
import datetime
import pickle
import os
import numpy as np
import sys
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import timezone
import string
from string import punctuation
from langdetect import detect
from scipy.sparse import hstack
from scipy import sparse
from xgboost import XGBClassifier
from pytz import utc, timezone

In [ ]:
df = pd.read_pickle('modeling_df_final.pkl')

In [ ]:
df.dtypes

In [4]:
df['text_cleaned'] = df['text_cleaned'].str.replace('aa', '')
df['text_cleaned'] = df['text_cleaned'].str.replace('aaa', '')

In [6]:
modeling_df = df.groupby(['user_id','is_bot']).agg({'timestamp':['min', 'max'],
                                                    'followers':'mean',
                                                    'following':'mean',
                                                    'is_retweet':'sum',
                                                    'is_quote_tweet':'sum',
                                                    'upper_count':'sum', 
                                                    'char_count':'sum',
                                                    'word_count':'sum', 
                                                    'hashtag_count':'sum',
                                                    'text_cleaned': lambda x: ' '.join(x),
                                                    'screen_name':'count'}).reset_index()
modeling_df.columns = modeling_df.columns.droplevel(1)
modeling_df['posts_during_period'] = modeling_df['screen_name']
modeling_df.drop(['timestamp','screen_name'], axis = 1, inplace = True)
modeling_df['ratio_old'] = modeling_df['followers']/(modeling_df['following']+0.0000001)
modeling_df['ratio'] = modeling_df['ratio_old'].fillna(0)
modeling_df.drop(['ratio_old'], axis = 1, inplace = True)
modeling_df['ratio'] = round(modeling_df['ratio'],2)
modeling_df['average_char'] = modeling_df['char_count']/modeling_df['posts_during_period']

In [18]:
count_vect = TfidfVectorizer(min_df = 4, max_df = .4, 
                             max_features = 35000,
                             ngram_range = (1,2),
                            stop_words = 'english') 
df_text = count_vect.fit_transform(modeling_df['text_cleaned'])

In [19]:
df_text.shape

(76035, 35000)

In [14]:
X_sparse = modeling_df.drop(['user_id','text_cleaned','is_quote_tweet'], axis = 1)
text_features = count_vect.get_feature_names()
number_features = list(X_sparse.columns)
feature_names = text_features + number_features

In [15]:
X_sparse = sparse.csr_matrix(X_sparse.values.astype(np.float))

In [16]:
X_sparse.shape

(76035, 11)

In [20]:
X_final = hstack((X_sparse, df_text))
del(df_text, X_sparse)
y = modeling_df['is_bot']
X_train, X_test, y_train, y_test = train_test_split(X_final, y, 
                                                    test_size = 0.2, random_state = 42)

# Text RF Model

In [23]:
rf_model = RandomForestClassifier(n_estimators=5,random_state = 44) 
rf_model.fit(X_train, y_train)
pred = rf_model.predict(X_test)
print(accuracy_score(y_test,pred))
print(precision_score(y_test,pred))
print(f1_score(y_test,pred))

0.9978299467350562
0.9558823529411765
0.8873720136518771


In [24]:
recall_score(y_test,pred)

0.8280254777070064

In [25]:
pred.sum()

136

In [26]:
y_test.sum()

157

In [27]:
feats = {} 
for feature, importance in zip(feature_names, rf_model.feature_importances_):
    feats[feature] = importance
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances = importances.sort_values(by='Gini-importance',ascending = False)

In [28]:
importances.head(10)

,Gini-importance
ab,0.132785
tcot tgdn,0.043439
cristiano,0.030495
isis terrorists,0.028187
jake tapper,0.027911
amp let,0.027130
ultra,0.026360
kode,0.020161
wholesome,0.019869
abbey,0.019045


# Logistic Model

In [29]:
log_model = LogisticRegression()
log_model.fit(X_train, y_train)
f1_score(y_test,log_model.predict(X_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.3350253807106599

In [30]:
f1_score(y_test,log_model.predict(X_test))

0.3350253807106599

In [31]:
recall_score(y_test,log_model.predict(X_test))

0.21019108280254778

In [ ]:
#modeling_df.to_pickle('modeling_df_final.pkl')

# Naive Bayes

In [32]:
clf = MultinomialNB().fit(X_train, y_train)
# X_test_counts = count_vect.transform(X_test[:,4])
# X_test_tfidf = tfidf_transformer.transform(X_test_counts)
f1_score(y_test, clf.predict(X_test))

0.5809768637532133

# Gradient Boosting

In [33]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
print(f1_score(y_test, xgb_model.predict(X_test)))
print(recall_score(y_test, xgb_model.predict(X_test)))

1.0
1.0


In [35]:
feats = {} 
for feature, importance in zip(feature_names, xgb_model.feature_importances_):
    feats[feature] = importance
    
importances2 = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances2 = importances2.sort_values(by='Gini-importance',ascending = False)

In [36]:
importances2.head(20)

,Gini-importance
ab,0.991253
trump assassinated,0.003931
abbott,0.002916
ab workout,0.001900
word_count,0.000000
promotes,0.000000
promo,0.000000
promo click,0.000000
promo code,0.000000
promos,0.000000
